In [1]:
try:
    import nltk
    nltk.download('averaged_perceptron_tagger_eng')
except:
    !pip install nltk
    import nltk
    nltk.download('averaged_perceptron_tagger_eng')

!pip install tensorflow_hub

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/tom/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
try:
    import textattack
except:
    !pip install textattack

from textattack.attack_recipes import TextFoolerJin2019
from textattack.datasets import Dataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 1) Load model + tokenizer
model_name = "textattack/roberta-base-imdb"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# device = torch.device('cpu')

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model = model.to('cpu')
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# 2) Build the TextFooler attack
attack = TextFoolerJin2019.build(model_wrapper)

# 3) Create a tiny dataset with your sentence - Alternative approach
# Create examples as dictionaries to be more explicit
examples = [("The movie was absolutely wonderful!", 1)]  

# Create dataset with explicit input columns
dataset = Dataset(examples, input_columns=["text"])

# 4) Run the attack
attack_args = textattack.AttackArgs(num_examples=1, log_to_csv=None)
attacker = textattack.Attacker(attack, dataset, attack_args)
results = attacker.attack_dataset()

# 5) Inspect the first result
for res in results:
    print("Original :", res.original_text)
    print("Adversarial:", res.perturbed_text)
    print("\nDetailed results:")
    print("Original result:", res.original_result)
    print("Perturbed result:", res.perturbed_result)
    print("Number of queries:", res.num_queries)
    print("Goal function result:", res.goal_function_result_str())
    
    # Additional debugging - show the actual predictions
    print("\nPrediction details:")
    print("Original prediction:", res.original_result.output)
    print("Adversarial prediction:", res.perturbed_result.output)
    break

/home/tom/miniconda3/envs/steering-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-16 14:28:03.749948: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-16 14:28:04.000983: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-16 14:28:04.209464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752672484.338573    1310 cuda_dnn.cc:8579] Unable to register 

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|          | 0/1 [00:00<?, ?it/s]2025-07-16 14:36:24.632561: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-07-16 14:36:35.428226: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
100%|██████████| 1/1 [08:55<00:00, 535.66s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1: 100%|██████████| 1/1 [09:00<00:00, 540.29s/it]

[[1 (100%)]] --> [[0 (58%)]]

The [[movie]] was absolutely [[wonderful]]!

The [[photographer]] was absolutely [[sumptuous]]!



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 40.0%  |
| Average num. words per input: | 5.0    |
| Avg num queries:              | 98.0   |
+-------------------------------+--------+
Original : <bound method AttackResult.original_text of <textattack.attack_results.successful_attack_result.SuccessfulAttackResult object at 0x75543b0db210>>
Adversarial: <bound method AttackResult.perturbed_text of <textattack.attack_results.successful_attack_result.SuccessfulAttackResult object at 0x75543b0db2

In [4]:
"""
Compact Adversarial Attack Demo
Demonstrates model brittleness vs human perception with minimal code
"""

try:
    import textattack
except ImportError:
    !pip install textattack

import re
import random
from typing import Dict, Any
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from textattack.attack_recipes import TextFoolerJin2019
from textattack.datasets import Dataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_args import AttackArgs
from textattack.attacker import Attacker

class CompactAdversarialDemo:
    """Minimal adversarial attack demonstration"""
    
    def __init__(self):
        self.load_models()
    
    def load_models(self):
        """Load sentiment and toxicity models"""
        print("Loading models...")
        
        # Sentiment model
        self.sentiment_tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-imdb")
        self.sentiment_model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-imdb")
        self.sentiment_wrapper = HuggingFaceModelWrapper(self.sentiment_model, self.sentiment_tokenizer)
        
        # Toxicity model
        try:
            self.toxicity_pipeline = pipeline("text-classification", model="unitary/toxic-bert")
        except:
            self.toxicity_pipeline = pipeline("text-classification", model="martin-ha/toxic-comment-model")
        
        print("Models loaded!")
    
    def attack_sentiment(self, text: str, label: int) -> Dict[str, Any]:
        """Attack sentiment classifier"""
        dataset = Dataset([(text, label)], input_columns=["text"])
        attack = TextFoolerJin2019.build(self.sentiment_wrapper)
        
        attack_args = AttackArgs(num_examples=1, log_to_csv=None)
        attacker = Attacker(attack, dataset, attack_args)
        results = attacker.attack_dataset()
        
        if results:
            result = results[0]
            return {
                'original': result.original_text,
                'adversarial': result.perturbed_text,
                'success': 'SUCCEEDED' in result.goal_function_result_str(),
                'orig_pred': result.original_result.output,
                'adv_pred': result.perturbed_result.output,
                # 'human_readable': self.assess_human_perception(result.original_text, result.perturbed_text)
            }
        return None
    
    def bypass_toxicity(self, toxic_text: str) -> Dict[str, Any]:
        """Try to bypass toxicity detection"""
        original_score = self.check_toxicity(toxic_text)
        
        # Token-level attacks
        attacks = {
            'punctuation': self.inject_punctuation(toxic_text),
            'whitespace': self.add_whitespace(toxic_text),
            'leetspeak': self.to_leetspeak(toxic_text),
            'unicode': self.substitute_unicode(toxic_text)
        }
        
        results = {'original': original_score}
        
        for method, modified_text in attacks.items():
            modified_score = self.check_toxicity(modified_text)
            results[method] = {
                'text': modified_text,
                'bypassed': original_score['is_toxic'] and not modified_score['is_toxic'],
                'score': modified_score
            }
        
        return results
    
    def check_toxicity(self, text: str) -> Dict[str, Any]:
        """Check if text is toxic"""
        try:
            result = self.toxicity_pipeline(text)[0]
            return {
                'is_toxic': result['label'].upper() in ['TOXIC', 'TOXICITY', '1'],
                'confidence': result['score'],
                'label': result['label']
            }
        except Exception as e:
            return {'is_toxic': False, 'confidence': 0.0, 'label': 'ERROR', 'error': str(e)}
    
    def inject_punctuation(self, text: str) -> str:
        """Add punctuation to bypass detection"""
        words = text.split()
        if words:
            idx = random.randint(0, len(words) - 1)
            word = words[idx]
            if len(word) > 2:
                pos = random.randint(1, len(word) - 1)
                words[idx] = word[:pos] + '.' + word[pos:]
        return ' '.join(words)
    
    def add_whitespace(self, text: str) -> str:
        """Add extra whitespace"""
        return re.sub(r' ', '  ', text)
    
    def to_leetspeak(self, text: str) -> str:
        """Convert to leetspeak"""
        leet_map = {'a': '4', 'e': '3', 'i': '1', 'o': '0', 's': '5'}
        result = text.lower()
        for char, replacement in leet_map.items():
            if random.random() < 0.5:
                result = result.replace(char, replacement)
        return result
    
    def substitute_unicode(self, text: str) -> str:
        """Substitute with similar Unicode characters"""
        subs = {'a': 'а', 'e': 'е', 'o': 'о', 'i': 'і'}  # Cyrillic lookalikes
        result = text.lower()
        for char, replacement in subs.items():
            if char in result and random.random() < 0.3:
                result = result.replace(char, replacement, 1)
        return result
    
    def run_demo(self):
        """Run complete demonstration"""
        print("=" * 60)
        print("ADVERSARIAL ATTACK DEMONSTRATION")
        print("=" * 60)
        
        # 2. Toxicity Bypass Demo
        print("\n\n2. TOXICITY DETECTION BYPASS")
        print("-" * 40)
        
        toxic_text = "You are not intelligent and show no motivations."
        toxicity_results = self.bypass_toxicity(toxic_text)
        
        print(f"ORIGINAL: '{toxic_text}'")
        print(f"ORIGINAL TOXICITY: {toxicity_results['original']['label']} ({toxicity_results['original']['confidence']:.3f})")
        
        print("\nBYPASS ATTEMPTS:")
        for method, result in toxicity_results.items():
            if method != 'original':
                status = "✓ SUCCESS" if result['bypassed'] else "✗ FAILED"
                print(f"{status} - {method}: '{result['text']}'")
                print(f"  Toxicity: {result['score']['label']} ({result['score']['confidence']:.3f})")
        
        successful_bypasses = sum(1 for method, result in toxicity_results.items() 
                                if method != 'original' and result['bypassed'])
        total_attempts = len(toxicity_results) - 1
        
        print(f"\nTOXICITY BYPASS SUCCESS RATE: {successful_bypasses}/{total_attempts} ({successful_bypasses/total_attempts*100:.1f}%)")
        print("\n" + "=" * 60)

def main():
    """Run the compact demo"""
    demo = CompactAdversarialDemo()
    demo.run_demo()

if __name__ == "__main__":
    main()

Loading models...


Some weights of the model checkpoint at textattack/roberta-base-imdb were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Models loaded!
ADVERSARIAL ATTACK DEMONSTRATION


2. TOXICITY DETECTION BYPASS
----------------------------------------
ORIGINAL: 'You are not intelligent and show no motivations.'
ORIGINAL TOXICITY: toxic (0.524)

BYPASS ATTEMPTS:
✗ FAILED - punctuation: 'You a.re not intelligent and show no motivations.'
  Toxicity: toxic (0.403)
✗ FAILED - whitespace: 'You  are  not  intelligent  and  show  no  motivations.'
  Toxicity: toxic (0.524)
✗ FAILED - leetspeak: 'you 4re not 1ntell1gent 4nd show no mot1v4t1ons.'
  Toxicity: toxic (0.627)
✗ FAILED - unicode: 'you are not іntelligent and show no motivations.'
  Toxicity: toxic (0.080)

TOXICITY BYPASS SUCCESS RATE: 0/4 (0.0%)

